In [51]:
import pandas as pd
transfers = pd.read_parquet("s3://prm-gp2gp-data-sandbox-dev/transfers-sample-3/2-2021-transfers.parquet")
total_count = len(transfers)
print("Total transfers:", total_count)
print("First transfer:", min(transfers["date_requested"]))
print("Last transfer:", max(transfers["date_requested"]))

def pct(num):
    return "(" + str(round(num * 100, 2)) + "%)"

print("\nHigh level breakdown:")
failed = transfers["status"] == "FAILED"
pending = transfers["status"] == "PENDING"
pending_with_error = transfers["status"] == "PENDING_WITH_ERROR"
integrated = transfers["status"] == "INTEGRATED"
failed_count = len(transfers[failed])
pending_count = len(transfers[pending])
pending_with_error_count = len(transfers[pending_with_error])
integrated_count =  len(transfers[integrated])
print("Failed:", failed_count, pct(failed_count / total_count))
print("Pending:", pending_count, pct(pending_count / total_count))
print("Pending with error:", pending_with_error_count, pct(pending_with_error_count / total_count))
print("Integrated:", integrated_count, pct(integrated_count / total_count))

eight_days_in_seconds = 691200
integrated = transfers["status"] == "INTEGRATED"
within_8_days = transfers["sla_duration"] <= eight_days_in_seconds
beyond_8_days = transfers["sla_duration"] > eight_days_in_seconds
within_8_days_count = len(transfers[integrated & within_8_days])
beyond_8_days_count = len(transfers[integrated & beyond_8_days])
print("\nBreakdown of integrated:")
print("Integrated within 8 days:", within_8_days_count, pct(within_8_days_count / total_count))
print("Integrated beyond 8 days:", beyond_8_days_count, pct(beyond_8_days_count / total_count))

not_integrated = transfers["status"] != "INTEGRATED"
fallback_count = len(transfers[(integrated & beyond_8_days) | not_integrated])
print("\nPaper fallback:", fallback_count, pct(fallback_count / total_count))


Total transfers: 234557
First transfer: 2021-02-01 05:59:00.037000
Last transfer: 2021-02-28 23:04:58.544000

High level breakdown:
Failed: 4237 (1.81%)
Pending: 7872 (3.36%)
Pending with error: 4452 (1.9%)
Integrated: 217996 (92.94%)

Breakdown of integrated:
Integrated within 8 days: 207988 (88.67%)
Integrated beyond 8 days: 10008 (4.27%)

Paper fallback: 26569 (11.33%)
